In [43]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors

In [20]:
df_all_nba = pd.read_csv('data/all_nba_expanded.csv')
df_draft = pd.read_csv('data/combine_stats_trimmed.csv')

In [21]:
all_nba_players = set(df_all_nba['Player Name'])

In [22]:
df_draft['year'] = df_draft['year'].apply(lambda x: '2024-25' if '2024-25' in x else x)

In [23]:
filtered_df_draft = df_draft[(df_draft['player'].isin(all_nba_players)) | (df_draft['year'] == '2024-25')]

In [24]:
filtered_df_draft

,Unnamed: 0,player,pos,year,bodyfat%,hand_len_in,hand_width_in,height_wo_shoes,height_w_shoes,standing_reach,weight_lbs,wingspan,lane_agility_s,shuttle_run_s,three_q_sprint_s,standing_vert_in,max_vert_in,max_bench_reps
49,49,Michael Redd,SG-SF,2000-01,NaN,NaN,NaN,76.75,NaN,102.0,214.0,81.00,12.06,NaN,3.34,27.0,31.0,12.0
66,66,Gilbert Arenas,SG,2001-02,5.3,NaN,NaN,74.25,NaN,99.5,199.0,81.50,NaN,NaN,3.25,31.5,36.0,12.0
79,79,Tyson Chandler,SF-PF,2001-02,6.7,NaN,NaN,83.50,NaN,110.0,224.0,87.00,12.13,NaN,3.36,31.0,33.5,2.0
112,112,Joe Johnson,SF-SG,2001-02,6.7,NaN,NaN,78.75,NaN,105.0,226.0,81.00,12.05,NaN,3.40,32.5,36.5,12.0
150,150,Carlos Boozer,PF,2002-03,NaN,NaN,NaN,79.75,NaN,108.5,258.0,86.25,10.77,NaN,3.31,26.5,28.5,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1657,1657,Ja'Kobe Walter,SG,2024-25,NaN,8.75,9.75,76.25,NaN,102.5,197.6,82.00,11.00,2.92,3.04,30.5,38.5,NaN
1658,1658,Kel'el Ware,C,2024-25,NaN,9.50,10.25,83.75,NaN,112.5,230.0,88.50,10.97,2.91,3.29,32.5,36.0,NaN
1659,1659,Jamir Watkins,SF,2024-25,NaN,9.00,9.50,77.00,NaN,104.0,211.6,82.75,10.64,3.16,3.01,30.5,35.5,NaN
1660,1660,Jaylen Wells,SF,2024-25,NaN,8.25,9.25,78.50,NaN,103.5,206.2,79.25,11.28,3.03,3.08,27.5,36.0,NaN


In [29]:
numeric_cols = filtered_df_draft.select_dtypes(include=['number']).columns
column_means = filtered_df_draft[numeric_cols].mean()
filtered_df_draft[numeric_cols] = filtered_df_draft[numeric_cols].fillna(column_means)
filtered_df_draft

C:\Users\Pazin_2\AppData\Local\Temp\ipykernel_30812\1658296732.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_draft[numeric_cols] = filtered_df_draft[numeric_cols].fillna(column_means)


,Unnamed: 0,player,pos,year,bodyfat%,hand_len_in,hand_width_in,height_wo_shoes,height_w_shoes,standing_reach,weight_lbs,wingspan,lane_agility_s,shuttle_run_s,three_q_sprint_s,standing_vert_in,max_vert_in,max_bench_reps
49,49,Michael Redd,SG-SF,2000-01,7.354167,8.75,9.44213,76.75,78.806818,102.0,214.0,81.00,12.060000,3.030357,3.34,27.0,31.0,12.000000
66,66,Gilbert Arenas,SG,2001-02,5.300000,8.75,9.44213,74.25,78.806818,99.5,199.0,81.50,11.270403,3.030357,3.25,31.5,36.0,12.000000
79,79,Tyson Chandler,SF-PF,2001-02,6.700000,8.75,9.44213,83.50,78.806818,110.0,224.0,87.00,12.130000,3.030357,3.36,31.0,33.5,2.000000
112,112,Joe Johnson,SF-SG,2001-02,6.700000,8.75,9.44213,78.75,78.806818,105.0,226.0,81.00,12.050000,3.030357,3.40,32.5,36.5,12.000000
150,150,Carlos Boozer,PF,2002-03,7.354167,8.75,9.44213,79.75,78.806818,108.5,258.0,86.25,10.770000,3.030357,3.31,26.5,28.5,13.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1657,1657,Ja'Kobe Walter,SG,2024-25,7.354167,8.75,9.75000,76.25,78.806818,102.5,197.6,82.00,11.000000,2.920000,3.04,30.5,38.5,10.272727
1658,1658,Kel'el Ware,C,2024-25,7.354167,9.50,10.25000,83.75,78.806818,112.5,230.0,88.50,10.970000,2.910000,3.29,32.5,36.0,10.272727
1659,1659,Jamir Watkins,SF,2024-25,7.354167,9.00,9.50000,77.00,78.806818,104.0,211.6,82.75,10.640000,3.160000,3.01,30.5,35.5,10.272727
1660,1660,Jaylen Wells,SF,2024-25,7.354167,8.25,9.25000,78.50,78.806818,103.5,206.2,79.25,11.280000,3.030000,3.08,27.5,36.0,10.272727


In [34]:
numeric_cols = filtered_df_draft.select_dtypes(include=['number']).columns[1:]
numeric_cols

Index(['bodyfat%', 'hand_len_in', 'hand_width_in', 'height_wo_shoes',
       'height_w_shoes', 'standing_reach', 'weight_lbs', 'wingspan',
       'lane_agility_s', 'shuttle_run_s', 'three_q_sprint_s',
       'standing_vert_in', 'max_vert_in', 'max_bench_reps'],
      dtype='object')

In [36]:
scaler = StandardScaler()
filtered_df_draft[numeric_cols] = scaler.fit_transform(filtered_df_draft[numeric_cols])
train_df = filtered_df_draft[filtered_df_draft['year'] != '2024-25']
test_df = filtered_df_draft[filtered_df_draft['year'] == '2024-25']

C:\Users\Pazin_2\AppData\Local\Temp\ipykernel_30812\1557180870.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_draft[numeric_cols] = scaler.fit_transform(filtered_df_draft[numeric_cols])


In [39]:
X_train = train_df[numeric_cols]
X_test = test_df[numeric_cols]

In [44]:
neigh = NearestNeighbors(n_neighbors=5)
neigh.fit(X_train)
distances, indices = neigh.kneighbors(X_test)

In [46]:
for i, (distance, index) in enumerate(zip(distances, indices)):
    # Retrieve the test player name
    test_player_name = test_df.iloc[i]['player']
    print(f"\nDraft Player: {test_player_name}:")

    # Iterate over each set of indices and distances for the nearest neighbors
    for dist, idx in zip(distance, index):
        # Retrieve the neighbor's details from the training set
        neighbor_player_name = train_df.iloc[idx]['player']
        print(f"{neighbor_player_name} Distance {dist:.2f}")


Draft Player: Michael Ajayi:
LeBron James Distance 2.89
Carmelo Anthony Distance 3.18
Michael Redd Distance 3.26
Chris Bosh Distance 3.27
Pascal Siakam Distance 3.40

Draft Player: Melvin Ajinca:
Michael Redd Distance 1.82
Chris Bosh Distance 3.00
LeBron James Distance 3.11
Joe Johnson Distance 3.12
Klay Thompson Distance 3.36

Draft Player: Trey Alexander:
Michael Redd Distance 2.79
Gilbert Arenas Distance 3.16
Klay Thompson Distance 3.21
Carmelo Anthony Distance 3.46
Dwyane Wade Distance 3.46

Draft Player: Izan Almansa:
LeBron James Distance 2.42
Amar'e Stoudemire Distance 2.58
Carmelo Anthony Distance 2.85
David Lee Distance 2.89
Anthony Davis Distance 3.02

Draft Player: Reece Beekman:
Dwyane Wade Distance 2.48
John Wall Distance 2.93
Russell Westbrook Distance 2.97
Glen Rice Distance 3.37
Gilbert Arenas Distance 3.38

Draft Player: Adem Bona:
Amar'e Stoudemire Distance 3.07
LeBron James Distance 3.53
Pascal Siakam Distance 3.66
Carmelo Anthony Distance 3.94
Dwyane Wade Distance 